In [ ]:
whatshap phase --indels --max-coverage 15 --ignore-read-groups -o HuRef.chr22.phased.vcf HuRef_SNPs.22.final.vcf HuRef.chr22.bam

In [ ]:
HapCUT2/build/extractHAIRS --bam whatshap/HuRef.chr22.bam --VCF whatshap/HuRef_SNPs.22.final.vcf --out whatshap/HuRef_SNPs.22.fragment_file

In [ ]:
HapCUT2/build/HAPCUT2 --fragments whatshap/HuRef_SNPs.22.fragment_file --vcf whatshap/HuRef_SNPs.22.final.vcf --out whatshap/HuRef.chr22.phased.HAPCUT2.txt

In [ ]:
whatshap hapcut2vcf -o whatshap/HuRef.chr22.phased.HAPCUT2.vcf whatshap/HuRef_SNPs.22.final.vcf whatshap/HuRef.chr22.phased.HAPCUT2.txt

In [13]:
for i in range(1,23):
    
    print (i)
    
    os.system('HapCUT2/build/extractHAIRS --bam whatshap/HuRef.chr'+str(i)+'.bam '+ \
        '--VCF whatshap/HuRef_SNPs.'+str(i)+'.final.vcf '+ \
        '--out whatshap/HuRef_SNPs.'+str(i)+'.fragment_file')
    
    os.system('HapCUT2/build/HAPCUT2 --fragments whatshap/HuRef_SNPs.'+str(i)+'.fragment_file '+ \
        '--vcf whatshap/HuRef_SNPs.'+str(i)+'.final.vcf '+ \
        '--out whatshap/HuRef.chr'+str(i)+'.phased.HAPCUT2.txt')
    
    os.system('whatshap hapcut2vcf -o whatshap/HuRef.chr'+str(i)+'.phased.HAPCUT2.vcf '+ 
        'whatshap/HuRef_SNPs.'+str(i)+'.final.vcf ' + \
        'whatshap/HuRef.chr'+str(i)+'.phased.HAPCUT2.txt')

In [21]:
import glob
fls = glob.glob('whatshap/*.phased.vcf')

#from fnmatch import fnmatch
#[fl for fl in os.listdir('whatshap/.') if fnmatch(fl, '*.phased.vcf')]

for fl in fls:
    prefix = '.'.join(fl.split('.')[:-1])
    os.rename(fl, prefix+'.whatshap.vcf')

In [477]:
from collections import OrderedDict
hap = OrderedDict()

with open('whatshap/Phased_haplotypes.final.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        hap[lst[0]+':'+lst[1]] = lst[-1]

In [480]:
### recode

fls = glob.glob('whatshap/*.phased.whatshap.vcf')

for fl in fls:
    prefix = '.'.join(fl.split('.')[:-1])
    
    fh = open(prefix+'.recode.vcf', 'wt')
    fh.write('\t'.join(['chr','pos','hap1','hap2','block'])+'\n')

    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith('#'):
                continue
            
            lst = line.split('\t')
        
            pos = lst[0]+':'+lst[1]
        
            if lst[-1] == '0/1':
                fh.write(lst[0]+'\t'+lst[1]+'\tNA\tNA\tNA\n')
            
            else:
                block = lst[-1].split(':')[-1]
                hap1 = lst[-1].split(':')[0].split('|')[0]
            
                if hap1 == hap[pos]:
                    fh.write('\t'.join(lst[:2]+['1','0',block])+'\n')
                else:
                    fh.write('\t'.join(lst[:2]+['0','1',block])+'\n')
                
    fh.close()

In [113]:
hapBlock = OrderedDict()
hapLen = OrderedDict()
tot = -1

with open('whatshap/HuRef.chr4.phased.whatshap.recode.vcf') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        tot += 1
        
        if lst[-1] == 'NA' or lst[-1] == 'block':
            continue
        
        if lst[-1] not in hapBlock.keys():
            hapBlock[lst[-1]] = []
            hapLen[lst[-1]] = []
        hapBlock[lst[-1]].append(lst[2])
        hapLen[lst[-1]].append(int(lst[1]))

In [114]:
tot

130152

In [115]:
hapBlock['3793818']

KeyError: '3793818'

In [85]:
Counter(hapBlock['3793818']).most_common(1)[0][1]/len(hapBlock['3793818'])

1.0

In [97]:
Counter(['0', '0', '0', '0', '0', '1', '1']).most_common(2)[1][1]

2

In [116]:
from collections import Counter

cmplt = 0
i = 0
for ke, val in hapBlock.items():
    cmplt += len(val)
    ratio = Counter(val).most_common(1)[0][1]/len(val)
    if (ratio < 1):
        i += Counter(val).most_common(2)[1][1]
        print (i)
        
        #i += 1
        #print (ke+'\t'+str(ratio))

1
4
5
6
7
8
9
10
11
12
13
17
18
19
21
22
23
24
25
26
28
29
30
31
32
33
35
36
37
39
47
48
49
50
51
52
53
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
76
77
82
83
84
86
88
89
93
94
95
96
98
99
100
102
103
105
111
112
113
114
115
116
117
118
119
120
121
143
144
146
147
150
151
153
155
156
157
158
159
163
164
165
166
167
168
170
172
173
174
176
177
178
179
180
182
183
188
189
190
195
196
1049
1052
1053
1054
1056
1057
1058
1060
1062
1063
1064
1066
1068
1075
1076
1079
1081
1082
1084
1086
1087
1088
1089
1090
1092
1093
1094
1095
1096
1097
1104
1105
1115
1354
1418
1419
1421
1423
1425


In [62]:
len(hapBlock)

24248

In [76]:
i

331

In [71]:
cmplt

85015

In [102]:
cmplt/tot

0.005789392537773667

In [82]:
maxBlock = 0

for ke, val in hapLen.items():
    l = val[-1] - val[0]
    
    if l > maxBlock:
        maxBlock = l

In [84]:
maxBlock/1000000

98.861243

In [77]:
### Percentage of phased single nucleotide variants (SNVs)

cmplt/tot



### Completeness

0.6553731113166821

In [ ]:
### Switch error rate (SER)


In [111]:
i = 0
with open('Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr4.haplotype.all.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split()
        
        if lst[1] == '1':
            i += 1

In [112]:
i

513

In [118]:
fls = glob.glob('whatshap/*.phased.whatshap.recode.vcf')
fls

['whatshap/HuRef.chr2.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr16.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr1.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr19.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr17.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr10.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr11.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr3.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr12.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr6.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr22.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr7.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr8.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr18.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr21.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr9.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr13.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr4.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr14.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr

In [119]:
sorted(fls, key = lambda x:int(x.split('.')[1].replace('chr', '')))

['whatshap/HuRef.chr1.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr2.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr3.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr4.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr5.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr6.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr7.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr8.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr9.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr10.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr11.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr12.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr13.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr14.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr15.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr16.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr17.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr18.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr19.phased.whatshap.recode.vcf',
 'whatshap/HuRef.chr2

In [ ]:
####################################################

In [485]:
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('whatshap/*.3.phased.whatshap.recode.vcf')
fls = sorted(fls, key = lambda x:int(x.split('.')[1].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[1]
    
    print (format(chrom, '=^50'))
    
    hapBlock = OrderedDict()
    hapPos = OrderedDict()
    numberOfSNPs = -1

    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
        
            numberOfSNPs += 1
        
            if lst[-1] == 'NA' or lst[-1] == 'block':
                continue
        
            if lst[-1] not in hapBlock.keys():
                hapBlock[lst[-1]] = []
                hapPos[lst[-1]] = []
            hapBlock[lst[-1]].append(lst[2])
            hapPos[lst[-1]].append(int(lst[1]))
    
    
    numberOfSegments = len(hapBlock)
    
    phasedSNPs = 0
    
    mismatch = switchError = hammingDistance = 0
    
    #hapChr = []
    for ke, val in hapBlock.items():
        phasedSNPs += len(val)
        
        if len(Counter(val))==2:
            hammingDistance += Counter(val).most_common(2)[1][1]
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        #hapChr.append(len(val))
        
        v1 = val[1:]
        v2 = val[:-1]
        
        vd = [a != b for a, b in zip(v1,v2)]
        
        switchError += sum(vd)

        if len(vd)==1:
            continue
                
        indexV = [i+1 for i in range(len(vd)) if vd[i]==True]
        indexV1 = indexV[1:]
        indexV2 = indexV[:-1]
        
        indexVd = [a - b for a, b in zip(indexV1,indexV2)]
        
        mismatch += indexVd.count(1)
        
        #indexV1.extend([len(val)])
        #indexV2.insert(0,0)
        #hapChr.extend([a - b for a, b in zip(indexV1, indexV2)])
        
    phasedPercent = round(phasedSNPs/numberOfSNPs*100,3)
        
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
    completeness = round((phasedSNPs-numberOfSegments)/numberOfSNPs*100,3)
    
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    #v1 = hapChr[1:]
    #v2 = hapChr[:-1]
    #error = sum([a != b for a, b in zip(v1,v2)])
    #ser = error/(cmplt)*100
    
    
    LHSLength = 0
    for ke, val in hapPos.items():
        l = val[-1] - val[0] + 1
            
        if l > LHSLength:
            LHSLength = l
            LHSVariant = len(val)
            
    LHSLength = round(LHSLength/1000000,3)
        
    for ke, val in hapPos.items():
        l = round((val[-1] - val[0] + 1)/1000000,3)
        
        if l == LHSLength:
            print (ke)
    
    
    MVP = 0
    for ke, val in hapPos.items():
        l = len(val)
            
        if l > MVP:
            MVP = l
    
    for ke, val in hapPos.items():
        if len(val) == MVP:
            print (ke)
    
    
    #QAN50 = 0
    #for size in sorted(hapChr, reverse=True):
    #    QAN50 += size
        #print ('largest %s, %s' %(size, QAN50))
    
    
        #if QAN50 > phased/2:
            #QAN50 = size
            #print ('QAN %s' %size)
            #break
    
        #ratio = Counter(val).most_common(1)[0][1]/len(val)
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        
    metrics[chrom] = [numberOfSNPs, phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs)
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()

=======================chr1=======================
102324460
158489704
Total number of SNPs: 129720
Number of phased SNVs: 37215
Percentage of phased SNVs: 28.689
Number of phased blocks: 13510
Completeness (NC): 18.274
Number of switch errors: 384
Switch error rate: 1.620000
Blockwise Hamming distance: 369
Blockwise Hamming rate: 0.992000
Largest haplotype segment: 143.915M
Most-variants-phased (MVP) segment: 103

=======================chr2=======================
63154816
117204657
Total number of SNPs: 141669
Number of phased SNVs: 40655
Percentage of phased SNVs: 28.697
Number of phased blocks: 14867
Completeness (NC): 18.203
Number of switch errors: 325
Switch error rate: 1.260000
Blockwise Hamming distance: 326
Blockwise Hamming rate: 0.802000
Largest haplotype segment: 118.066M
Most-variants-phased (MVP) segment: 77

=======================chr3=======================
53471338
103929183
Total number of SNPs: 127247
Number of phased SNVs: 38953
Percentage of phased SNVs: 30.612
Nu

In [486]:
fh = open('whatshap/metrics.whatshap.3.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()

In [487]:
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('whatshap/*.3.phased.HAPCUT2.recode.vcf')
fls = sorted(fls, key = lambda x:int(x.split('.')[1].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[1]
    
    print (format(chrom, '=^50'))
    
    hapBlock = OrderedDict()
    hapPos = OrderedDict()
    numberOfSNPs = -1

    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
        
            numberOfSNPs += 1
        
            if lst[-1] == 'NA' or lst[-1] == 'block':
                continue
        
            if lst[-1] not in hapBlock.keys():
                hapBlock[lst[-1]] = []
                hapPos[lst[-1]] = []
            hapBlock[lst[-1]].append(lst[2])
            hapPos[lst[-1]].append(int(lst[1]))
    
    
    numberOfSegments = len(hapBlock)
    
    phasedSNPs = 0
    
    mismatch = switchError = hammingDistance = 0
    
    #hapChr = []
    for ke, val in hapBlock.items():
        phasedSNPs += len(val)
        
        if len(Counter(val))==2:
            hammingDistance += Counter(val).most_common(2)[1][1]
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        #hapChr.append(len(val))
        
        v1 = val[1:]
        v2 = val[:-1]
        
        vd = [a != b for a, b in zip(v1,v2)]
        
        switchError += sum(vd)

        if len(vd)==1:
            continue
                
        indexV = [i+1 for i in range(len(vd)) if vd[i]==True]
        indexV1 = indexV[1:]
        indexV2 = indexV[:-1]
        
        indexVd = [a - b for a, b in zip(indexV1,indexV2)]
        
        mismatch += indexVd.count(1)
        
        #indexV1.extend([len(val)])
        #indexV2.insert(0,0)
        #hapChr.extend([a - b for a, b in zip(indexV1, indexV2)])
        
    phasedPercent = round(phasedSNPs/numberOfSNPs*100,3)
        
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
    completeness = round((phasedSNPs-numberOfSegments)/numberOfSNPs*100,3)
    
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    #v1 = hapChr[1:]
    #v2 = hapChr[:-1]
    #error = sum([a != b for a, b in zip(v1,v2)])
    #ser = error/(cmplt)*100
    
    
    LHSLength = 0
    for ke, val in hapPos.items():
        l = val[-1] - val[0] + 1
            
        if l > LHSLength:
            LHSLength = l
            LHSVariant = len(val)
            
    LHSLength = round(LHSLength/1000000,3)
        
    for ke, val in hapPos.items():
        l = round((val[-1] - val[0] + 1)/1000000,3)
        
        if l == LHSLength:
            print (ke)
        
        
    MVP = 0
    for ke, val in hapPos.items():
        l = len(val)
            
        if l > MVP:
            MVP = l
            
    for ke, val in hapPos.items():
        if len(val) == MVP:
            print (ke)
    
    
    #QAN50 = 0
    #for size in sorted(hapChr, reverse=True):
    #    QAN50 += size
        #print ('largest %s, %s' %(size, QAN50))
    
    
        #if QAN50 > phased/2:
            #QAN50 = size
            #print ('QAN %s' %size)
            #break
    
        #ratio = Counter(val).most_common(1)[0][1]/len(val)
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        
    metrics[chrom] = [numberOfSNPs, phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs)
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()

=======================chr1=======================
14320465
158497079
158503221
14320465
Total number of SNPs: 129720
Number of phased SNVs: 35224
Percentage of phased SNVs: 27.154
Number of phased blocks: 13049
Completeness (NC): 17.095
Number of switch errors: 272
Switch error rate: 1.227000
Blockwise Hamming distance: 238
Blockwise Hamming rate: 0.676000
Largest haplotype segment: 0.005M
Most-variants-phased (MVP) segment: 44

=======================chr2=======================
133047563
133047563
Total number of SNPs: 141669
Number of phased SNVs: 38598
Percentage of phased SNVs: 27.245
Number of phased blocks: 14427
Completeness (NC): 17.062
Number of switch errors: 246
Switch error rate: 1.018000
Blockwise Hamming distance: 246
Blockwise Hamming rate: 0.637000
Largest haplotype segment: 0.006M
Most-variants-phased (MVP) segment: 63

=======================chr3=======================
103937539
103937539
Total number of SNPs: 127247
Number of phased SNVs: 36968
Percentage of phased 

In [484]:
fh = open('whatshap/metrics.HAPCUT2.11.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()

In [456]:
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.*.haplotype.all.txt')
fls = sorted(fls, key = lambda x:int(x.split('.')[3].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[3]
    
    print (format(chrom, '=^50'))
    
    hapBlock = []
    hapPos = []
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            
            if line.startswith('hap1'):
                continue
            
            lst = line.split('\t')
            
            hapBlock.append(lst[1])
            hapPos.append(lst[0].split(':')[1])
            #hapPos.append(lst[0])
    
    
    numberOfSegments = 1
    numberOfSNPs = len(hapBlock)
    
    hapCount = Counter(hapBlock)
    
    phasedSNPs = numberOfSNPs-hapCount['7']
    completeness = round((phasedSNPs-1)/(numberOfSNPs-1)*100,3)
    
    phasedPercent = round(phasedSNPs/numberOfSNPs*100,3)
    
    hammingDistance = min(hapCount['0'], hapCount['1'])
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    LHSLength = round((int(hapPos[-1])-int(hapPos[0]))/1000000,3)
    MVP = phasedSNPs
    
    hapClean = [hap for hap in hapBlock if hap != '7']
    
    v1 = hapClean[1:]
    v2 = hapClean[:-1]
        
    vd = [a != b for a, b in zip(v1,v2)]
        
    switchError = sum(vd)
    
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
     
    metrics[chrom] = [numberOfSNPs, phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs)
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()

=======================chr1=======================
Total number of SNPs: 129720
Number of phased SNVs: 129241
Percentage of phased SNVs: 99.631
Number of phased blocks: 1
Completeness (NC): 99.631
Number of switch errors: 511
Switch error rate: 0.395000
Blockwise Hamming distance: 315
Blockwise Hamming rate: 0.244000
Largest haplotype segment: 248.393M
Most-variants-phased (MVP) segment: 129241

=======================chr2=======================
Total number of SNPs: 141669
Number of phased SNVs: 141225
Percentage of phased SNVs: 99.687
Number of phased blocks: 1
Completeness (NC): 99.687
Number of switch errors: 426
Switch error rate: 0.302000
Blockwise Hamming distance: 307
Blockwise Hamming rate: 0.217000
Largest haplotype segment: 242.959M
Most-variants-phased (MVP) segment: 141225

=======================chr3=======================
Total number of SNPs: 127247
Number of phased SNVs: 127069
Percentage of phased SNVs: 99.86
Number of phased blocks: 1
Completeness (NC): 99.86
Number 

In [457]:
fh = open('whatshap/metrics.Hapi.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()

In [458]:
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('Comparison/GenomeResearch/Consistency/Cell2.sperm11.*.haplotype.all.txt')
fls = sorted(fls, key = lambda x:int(x.split('.')[2].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[2]
    
    print (format(chrom, '=^50'))
    
    hapBlock = []
    hapPos = []
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            
            if line.startswith('hap1'):
                continue
            
            lst = line.split('\t')
            
            hapBlock.append(lst[1])
            #hapPos.append(lst[0].split(':')[1])
            hapPos.append(lst[0])
    
    
    numberOfSegments = 1
    numberOfSNPs = len(hapBlock)
    
    hapCount = Counter(hapBlock)
    
    phasedSNPs = numberOfSNPs-hapCount['7']
    completeness = round((phasedSNPs-1)/(numberOfSNPs-1)*100,3)
    
    phasedPercent = round(phasedSNPs/numberOfSNPs*100,3)
    
    hammingDistance = min(hapCount['0'], hapCount['1'])
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    LHSLength = round((int(hapPos[-1])-int(hapPos[0]))/1000000,3)
    MVP = phasedSNPs
    
    hapClean = [hap for hap in hapBlock if hap != '7']
    
    v1 = hapClean[1:]
    v2 = hapClean[:-1]
        
    vd = [a != b for a, b in zip(v1,v2)]
        
    switchError = sum(vd)
    
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
     
    metrics[chrom] = [numberOfSNPs, phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs)
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()

=======================chr1=======================
Total number of SNPs: 129720
Number of phased SNVs: 120708
Percentage of phased SNVs: 93.053
Number of phased blocks: 1
Completeness (NC): 93.053
Number of switch errors: 4623
Switch error rate: 3.830000
Blockwise Hamming distance: 6790
Blockwise Hamming rate: 5.625000
Largest haplotype segment: 248.393M
Most-variants-phased (MVP) segment: 120708

=======================chr2=======================
Total number of SNPs: 141669
Number of phased SNVs: 138930
Percentage of phased SNVs: 98.067
Number of phased blocks: 1
Completeness (NC): 98.067
Number of switch errors: 1298
Switch error rate: 0.934000
Blockwise Hamming distance: 1260
Blockwise Hamming rate: 0.907000
Largest haplotype segment: 242.959M
Most-variants-phased (MVP) segment: 138930

=======================chr3=======================
Total number of SNPs: 127247
Number of phased SNVs: 127067
Percentage of phased SNVs: 99.859
Number of phased blocks: 1
Completeness (NC): 99.859
N

In [459]:
fh = open('whatshap/metrics.PHMM.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()

In [ ]:
#######################

In [502]:
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)
nSAM = '10'

fls = glob.glob('whatshap/*.'+nSAM+'.phased.whatshap.recode.vcf')
fls = sorted(fls, key = lambda x:int(x.split('.')[1].replace('chr', '')))

numberOfSNPs = OrderedDict()

metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[1]
    
    print (format(chrom, '=^50'))
    
    hapBlock = OrderedDict()
    hapPos = OrderedDict()
    numberOfSNPs[chrom] = -1

    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
        
            numberOfSNPs[chrom] += 1
        
            if lst[-1] == 'NA' or lst[-1] == 'block':
                continue
        
            if lst[-1] not in hapBlock.keys():
                hapBlock[lst[-1]] = []
                hapPos[lst[-1]] = []
            hapBlock[lst[-1]].append(lst[2])
            hapPos[lst[-1]].append(int(lst[1]))
    
    
    numberOfSegments = len(hapBlock)
    
    phasedSNPs = 0
    
    mismatch = switchError = hammingDistance = 0
    
    #hapChr = []
    for ke, val in hapBlock.items():
        phasedSNPs += len(val)
        
        if len(Counter(val))==2:
            hammingDistance += Counter(val).most_common(2)[1][1]
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        #hapChr.append(len(val))
        
        v1 = val[1:]
        v2 = val[:-1]
        
        vd = [a != b for a, b in zip(v1,v2)]
        
        switchError += sum(vd)

        if len(vd)==1:
            continue
                
        indexV = [i+1 for i in range(len(vd)) if vd[i]==True]
        indexV1 = indexV[1:]
        indexV2 = indexV[:-1]
        
        indexVd = [a - b for a, b in zip(indexV1,indexV2)]
        
        mismatch += indexVd.count(1)
        
        #indexV1.extend([len(val)])
        #indexV2.insert(0,0)
        #hapChr.extend([a - b for a, b in zip(indexV1, indexV2)])
        
    phasedPercent = round(phasedSNPs/numberOfSNPs[chrom]*100,3)
        
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
    completeness = round((phasedSNPs-numberOfSegments)/numberOfSNPs[chrom]*100,3)
    
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    #v1 = hapChr[1:]
    #v2 = hapChr[:-1]
    #error = sum([a != b for a, b in zip(v1,v2)])
    #ser = error/(cmplt)*100
    
    
    LHSLength = 0
    for ke, val in hapPos.items():
        l = val[-1] - val[0] + 1
            
        if l > LHSLength:
            LHSLength = l
            LHSVariant = len(val)
            
    LHSLength = round(LHSLength/1000000,3)
        
    for ke, val in hapPos.items():
        l = round((val[-1] - val[0] + 1)/1000000,3)
        
        if l == LHSLength:
            print (ke)
    
    
    MVP = 0
    for ke, val in hapPos.items():
        l = len(val)
            
        if l > MVP:
            MVP = l
    
    for ke, val in hapPos.items():
        if len(val) == MVP:
            print (ke)
    
    
    #QAN50 = 0
    #for size in sorted(hapChr, reverse=True):
    #    QAN50 += size
        #print ('largest %s, %s' %(size, QAN50))
    
    
        #if QAN50 > phased/2:
            #QAN50 = size
            #print ('QAN %s' %size)
            #break
    
        #ratio = Counter(val).most_common(1)[0][1]/len(val)
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        
    metrics[chrom] = [numberOfSNPs[chrom], phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs[chrom])
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()
    
    
fh = open('whatshap/metrics.whatshap.'+nSAM+'.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()



###
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('whatshap/*.'+nSAM+'.phased.HAPCUT2.recode.vcf')
fls = sorted(fls, key = lambda x:int(x.split('.')[1].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[1]
    
    print (format(chrom, '=^50'))
    
    hapBlock = OrderedDict()
    hapPos = OrderedDict()
    #numberOfSNPs = -1

    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
        
            #numberOfSNPs += 1
        
            if lst[-1] == 'NA' or lst[-1] == 'block':
                continue
        
            if lst[-1] not in hapBlock.keys():
                hapBlock[lst[-1]] = []
                hapPos[lst[-1]] = []
            hapBlock[lst[-1]].append(lst[2])
            hapPos[lst[-1]].append(int(lst[1]))
    
    
    numberOfSegments = len(hapBlock)
    
    phasedSNPs = 0
    
    mismatch = switchError = hammingDistance = 0
    
    #hapChr = []
    for ke, val in hapBlock.items():
        phasedSNPs += len(val)
        
        if len(Counter(val))==2:
            hammingDistance += Counter(val).most_common(2)[1][1]
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        #hapChr.append(len(val))
        
        v1 = val[1:]
        v2 = val[:-1]
        
        vd = [a != b for a, b in zip(v1,v2)]
        
        switchError += sum(vd)

        if len(vd)==1:
            continue
                
        indexV = [i+1 for i in range(len(vd)) if vd[i]==True]
        indexV1 = indexV[1:]
        indexV2 = indexV[:-1]
        
        indexVd = [a - b for a, b in zip(indexV1,indexV2)]
        
        mismatch += indexVd.count(1)
        
        #indexV1.extend([len(val)])
        #indexV2.insert(0,0)
        #hapChr.extend([a - b for a, b in zip(indexV1, indexV2)])
        
    phasedPercent = round(phasedSNPs/numberOfSNPs[chrom]*100,3)
        
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
    completeness = round((phasedSNPs-numberOfSegments)/numberOfSNPs[chrom]*100,3)
    
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    #v1 = hapChr[1:]
    #v2 = hapChr[:-1]
    #error = sum([a != b for a, b in zip(v1,v2)])
    #ser = error/(cmplt)*100
    
    
    LHSLength = 0
    for ke, val in hapPos.items():
        l = val[-1] - val[0] + 1
            
        if l > LHSLength:
            LHSLength = l
            LHSVariant = len(val)
            
    LHSLength = round(LHSLength/1000000,3)
        
    for ke, val in hapPos.items():
        l = round((val[-1] - val[0] + 1)/1000000,3)
        
        if l == LHSLength:
            print (ke)
        
        
    MVP = 0
    for ke, val in hapPos.items():
        l = len(val)
            
        if l > MVP:
            MVP = l
            
    for ke, val in hapPos.items():
        if len(val) == MVP:
            print (ke)
    
    
    #QAN50 = 0
    #for size in sorted(hapChr, reverse=True):
    #    QAN50 += size
        #print ('largest %s, %s' %(size, QAN50))
    
    
        #if QAN50 > phased/2:
            #QAN50 = size
            #print ('QAN %s' %size)
            #break
    
        #ratio = Counter(val).most_common(1)[0][1]/len(val)
        #if (ratio < 1):
        #    i += Counter(val).most_common(2)[1][1]
        #print (i)
        
        
    metrics[chrom] = [numberOfSNPs[chrom], phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs[chrom])
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()
    
    
fh = open('whatshap/metrics.HAPCUT2.'+nSAM+'.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()






### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('whatshap/NEW5.Hapi2.sperm'+nSAM+'.chr*.haplotype.all.txt')
fls = sorted(fls, key = lambda x:int(x.split('.')[3].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[3]
    
    print (format(chrom, '=^50'))
    
    hapBlock = []
    hapPos = []
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            
            if line.startswith('hap1'):
                continue
            
            lst = line.split('\t')
            
            hapBlock.append(lst[1])
            hapPos.append(lst[0].split(':')[1])
            #hapPos.append(lst[0])
    
    
    numberOfSegments = 1
    #numberOfSNPs = len(hapBlock)
    
    hapCount = Counter(hapBlock)
    
    phasedSNPs = hapCount['0']+hapCount['1']
    completeness = round((phasedSNPs-1)/(numberOfSNPs[chrom]-1)*100,3)
    
    phasedPercent = round(phasedSNPs/numberOfSNPs[chrom]*100,3)
    
    hammingDistance = min(hapCount['0'], hapCount['1'])
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    LHSLength = round((int(hapPos[-1])-int(hapPos[0]))/1000000,3)
    MVP = phasedSNPs
    
    hapClean = [hap for hap in hapBlock if hap != '7']
    
    v1 = hapClean[1:]
    v2 = hapClean[:-1]
        
    vd = [a != b for a, b in zip(v1,v2)]
        
    switchError = sum(vd)
    
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
     
    metrics[chrom] = [numberOfSNPs[chrom], phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs[chrom])
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()

    
fh = open('whatshap/metrics.Hapi.'+nSAM+'.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()



###
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('whatshap/Cell2.sperm'+nSAM+'.chr*.haplotype.all.txt')
fls = sorted(fls, key = lambda x:int(x.split('.')[2].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[2]
    
    print (format(chrom, '=^50'))
    
    hapBlock = []
    hapPos = []
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            
            if line.startswith('hap1'):
                continue
            
            lst = line.split('\t')
            
            hapBlock.append(lst[1])
            #hapPos.append(lst[0].split(':')[1])
            hapPos.append(lst[0])
    
    
    numberOfSegments = 1
    #numberOfSNPs = len(hapBlock)
    
    hapCount = Counter(hapBlock)
    
    phasedSNPs = hapCount['0']+hapCount['1']
    completeness = round((phasedSNPs-1)/(numberOfSNPs[chrom]-1)*100,3)
    
    phasedPercent = round(phasedSNPs/numberOfSNPs[chrom]*100,3)
    
    hammingDistance = min(hapCount['0'], hapCount['1'])
    hammingRate = round(hammingDistance/phasedSNPs*100,3)
    
    LHSLength = round((int(hapPos[-1])-int(hapPos[0]))/1000000,3)
    MVP = phasedSNPs
    
    hapClean = [hap for hap in hapBlock if hap != '7']
    
    v1 = hapClean[1:]
    v2 = hapClean[:-1]
        
    vd = [a != b for a, b in zip(v1,v2)]
        
    switchError = sum(vd)
    
    SER = round(switchError/(phasedSNPs-numberOfSegments)*100,3)
     
    metrics[chrom] = [numberOfSNPs[chrom], phasedSNPs, phasedPercent, numberOfSegments,
                      completeness, switchError, SER, hammingDistance, hammingRate,
                      LHSLength, MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %numberOfSNPs[chrom])
    print ('Number of phased SNVs: %s' %phasedSNPs)
    print ('Percentage of phased SNVs: %s' %phasedPercent)
    print ('Number of phased blocks: %s' %numberOfSegments)
    print ('Completeness (NC): %s' %completeness)
    
    
    ### accuracy
    print ('Number of switch errors: %s' %switchError)
    print ('Switch error rate: %f' %SER)
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hammingDistance)
    print ('Blockwise Hamming rate: %f' %hammingRate)
    
    print ('Largest haplotype segment: %sM' %LHSLength)
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()
    

fh = open('whatshap/metrics.PHMM.'+nSAM+'.txt', 'wt')

fh.write('\t'.join(['numberOfSNPs','phasedSNPs','phasedPercent',
                   'numberOfSegments','completeness','switchError',
                   'SER','hammingDistance','hammingRate','LHSLength','MVP'])+'\n')

for ke, val in metrics.items():
    fh.write(ke+'\t'+'\t'.join(map(str,val))+'\n')
    
fh.close()

=======================chr1=======================
88770663
195037001
Total number of SNPs: 129720
Number of phased SNVs: 76142
Percentage of phased SNVs: 58.697
Number of phased blocks: 23095
Completeness (NC): 40.893
Number of switch errors: 616
Switch error rate: 1.161000
Blockwise Hamming distance: 631
Blockwise Hamming rate: 0.829000
Largest haplotype segment: 98.86M
Most-variants-phased (MVP) segment: 191

=======================chr2=======================
62307899
35870947
Total number of SNPs: 141669
Number of phased SNVs: 83960
Percentage of phased SNVs: 59.265
Number of phased blocks: 25571
Completeness (NC): 41.215
Number of switch errors: 489
Switch error rate: 0.837000
Blockwise Hamming distance: 595
Blockwise Hamming rate: 0.709000
Largest haplotype segment: 79.708M
Most-variants-phased (MVP) segment: 308

=======================chr3=======================
50847250
623978
Total number of SNPs: 127247
Number of phased SNVs: 77365
Percentage of phased SNVs: 60.799
Number of

In [400]:
fls = glob.glob('Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.*.haplotype.all.txt')
fls = sorted(fls, key = lambda x:int(x.split('.')[3].replace('chr', '')))
fls

['Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr1.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr2.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr3.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr4.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr5.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr6.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr7.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr8.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr9.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr10.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr11.haplotype.all.txt',
 'Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr12.haplotype

In [410]:
hapBlock = []
hapPos = []
with open('Comparison/GenomeResearch/Consistency/NEW5.Hapi2.sperm11.chr1.haplotype.all.txt') as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('hap1'):
            continue
        
        lst = line.split('\t')
            
        hapBlock.append(lst[1])
        hapPos.append(lst[0].split(':')[1])
    
tot = len(hapBlock)
hapCount = Counter(hapBlock)

In [414]:
tot

129720

In [413]:
hapCount['7']

479

In [424]:
min(hapCount['0'], hapCount['1'])

315

In [425]:
hapCount['0']

128926

In [434]:
fls = glob.glob('Comparison/GenomeResearch/Consistency/Cell2.sperm11.*.haplotype.all')
fls

[]

In [438]:
### Number of phased blocks
### Percentage of phased single nucleotide variants (SNVs)

fls = glob.glob('Comparison/GenomeResearch/Consistency/Cell2.sperm11.*.haplotype.all.txt')
fls = sorted(fls, key = lambda x:int(x.split('.')[2].replace('chr', '')))


metrics = OrderedDict()
for fl in fls:
    chrom = fl.split('.')[2]
    
    print (format(chrom, '=^50'))
    
    hapBlock = []
    hapPos = []
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            
            if line.startswith('hap1'):
                continue
            
            lst = line.split('\t')
            
            hapBlock.append(lst[1])
            #hapPos.append(lst[0].split(':')[1])
            hapPos.append(lst[0])
    
    
    tot = len(hapBlock)
    hapCount = Counter(hapBlock)
    phased = tot-hapCount['7']
    
    completeness = (phased-1)/(tot-1)
    
    hamming = min(hapCount['0'], hapCount['1'])
    
    LPSLength = (int(hapPos[-1])-int(hapPos[0]))/1000000
    MVP = phased
    
    hapClean = [hap for hap in hapBlock if hap != '7']
    
    v1 = hapClean[1:]
    v2 = hapClean[:-1]
        
    vd = [a != b for a, b in zip(v1,v2)]
        
    error = sum(vd)
    
    ser = error/(phased-1)*100
     
    metrics[chrom] = [tot, phased, round(phased/tot*100,3), round(completeness*100,3), 
                      len(hapBlock), error, round(ser,3), hamming, 
                      round(hamming/phased*100,3),round(LPSLength,3), MVP]
    
    ### completeness
    print ('Total number of SNPs: %s' %tot)
    print ('Number of phased SNVs: %s' %phased)
    print ('Percentage of phased SNVs: %s' %round(phased/tot,3))
    print ('Completeness (NC): %s' %round(completeness,3))
    
    ### accuracy
    print ('Number of switch errors: %s' %error)
    print ('Switch error rate: %f' %round(ser,3))
    #print ('Number of mismatch: %s' %mismatch)
    
    print ('Blockwise Hamming distance: %s' %hamming)
    print ('Blockwise Hamming rate: %f' %round(hamming/phased*100,3))
    
    print ('Largest haplotype segment: %sM' %round(LPSLength,3))
    print ('Most-variants-phased (MVP) segment: %s' %MVP)
    
    #print ('QAN %s' %QAN50)
    print ()

=======================chr1=======================
Total number of SNPs: 129720
Number of phased SNVs: 120708
Percentage of phased SNVs: 0.931
Completeness (NC): 0.931
Number of switch errors: 4623
Switch error rate: 3.830000
Blockwise Hamming distance: 6790
Blockwise Hamming rate: 5.625000
Largest haplotype segment: 248.393M
Most-variants-phased (MVP) segment: 120708

=======================chr2=======================
Total number of SNPs: 141669
Number of phased SNVs: 138930
Percentage of phased SNVs: 0.981
Completeness (NC): 0.981
Number of switch errors: 1298
Switch error rate: 0.934000
Blockwise Hamming distance: 1260
Blockwise Hamming rate: 0.907000
Largest haplotype segment: 242.959M
Most-variants-phased (MVP) segment: 138930

=======================chr3=======================
Total number of SNPs: 127247
Number of phased SNVs: 127067
Percentage of phased SNVs: 0.999
Completeness (NC): 0.999
Number of switch errors: 294
Switch error rate: 0.231000
Blockwise Hamming distance: 202

In [361]:
Counter(v).most_common(2)[1][1]

2

In [365]:
len(Counter(v))

1

In [381]:
phased

78948

In [380]:
sorted(hapChr, reverse=True)

[3682,
 427,
 347,
 280,
 238,
 201,
 167,
 156,
 139,
 132,
 131,
 122,
 121,
 113,
 108,
 108,
 94,
 93,
 91,
 89,
 85,
 84,
 78,
 77,
 74,
 72,
 69,
 68,
 67,
 64,
 63,
 62,
 61,
 61,
 59,
 57,
 56,
 55,
 54,
 53,
 51,
 51,
 51,
 51,
 50,
 50,
 48,
 48,
 47,
 46,
 46,
 44,
 44,
 44,
 44,
 44,
 44,
 42,
 41,
 41,
 41,
 40,
 40,
 40,
 40,
 40,
 40,
 39,
 39,
 39,
 39,
 38,
 38,
 38,
 38,
 37,
 36,
 36,
 36,
 36,
 36,
 36,
 36,
 35,
 35,
 35,
 34,
 34,
 34,
 34,
 34,
 34,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 33,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 32,
 31,
 31,
 31,
 31,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 30,
 29,
 29,
 29,
 29,
 28,
 28,
 28,
 28,
 27,
 27,
 27,
 27,
 27,
 27,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 26,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 24,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21

In [364]:
v = [0,0,0,0,0,0,0]

In [ ]:
C

In [302]:
v1 = v[1:]
v2 = v[:-1]

In [303]:
v1

[1, 0]

In [304]:
v2

[0, 1]

In [305]:
v = [a != b for a, b in zip(v1,v2)]

In [306]:
v

[True, True]

In [307]:
indexV = [i+1 for i in range(len(v)) if v[i]==True]

In [308]:
indexV1 = indexV[1:]
indexV2 = indexV[:-1]

In [297]:
indexVd = [a - b for a, b in zip(indexV1,indexV2)]
indexVd

[1]

In [312]:
indexV1.extend([len(v)])

In [313]:
indexV1

[2, 2]

In [314]:
indexV2.insert(0,0)

In [315]:
indexV2

[0, 1]

In [159]:
sum([a != b for a, b in zip(v1,v2)])

2

NameError: name 'a' is not defined

In [ ]:
*************************************************************************

###### Completeness
# % of phased SNVs

# sum the number of phase connections across all haplotype segments and divide
# by the maximum possible number of phase connections, which is equal to the
# number of heterozygous variants on a chromosome minus one.


###### Accuracy
# switch error rate
# blockwise hamming rate


###### Contiguity
# length of the largest haplotype segment

In [ ]:
###### Comparison of phasing strategies for whole human genomes

### Percentage of phased single nucleotide variants (SNVs) ***
    ### Number of phasing blocks
    ### Number of phased SNVs
    ### % phased SNPs (among golden set)
    
### Switch error rate (SER)
    ### SNPs used for switch error measurement
    ### Number of switch errors
    ### SER
    
### Haplotype block length : QAN50 - Completeness and accuracy


### Pairwise SNVs phasing accuracy and phasing yield
### Percentage of genes fully phased
### SER as a function of haplotype diversity
### Shared switch error sites





###### WhatsHap: fast and accurate read-based phasing

### phasing accuracy
### phasing completeness 




###### Dense and accurate whole-chromosome haplotyping of individual genomes

### Largest haplotype segment ***

### Largest haplotype segment Hamming rate

### Hamming error rate is equal to the fraction of wrongly phased heterozygous variants


# completeness (i.e., % of phased SNVs), 
# contiguity (length of the largest haplotype segment) ***
# accuracy (agreement with the “reference” standard) of each assembled haplotype.




In [330]:
hapBlock

OrderedDict([('16096825', ['1', '1']),
             ('16106307', ['0', '1', '1']),
             ('16191691', ['1', '1']),
             ('16228793', ['1', '1']),
             ('16874056', ['0', '0', '0', '1']),
             ('16876292', ['1', '1', '1']),
             ('16888538', ['1', '1', '1']),
             ('16890136', ['1', '1']),
             ('16894117', ['1', '0']),
             ('16906273', ['1', '1']),
             ('17066493', ['1', '1', '1']),
             ('17107266', ['0', '0']),
             ('17108394', ['1', '1']),
             ('17118076', ['1', '1', '1', '1', '1']),
             ('17122332', ['1', '1']),
             ('17130268', ['1', '1', '1']),
             ('17141339', ['1', '1']),
             ('17150676', ['1', '1']),
             ('17152611', ['1', '1']),
             ('17160807', ['1', '1', '1']),
             ('17169347', ['1', '1']),
             ('17169979', ['1', '1']),
             ('17175781', ['1', '1']),
             ('17177906', ['1', '1']),
         

[2,
 3,
 2,
 2,
 4,
 3,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 5,
 2,
 3,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 26,
 4,
 3,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 3,
 8,
 5,
 2,
 7,
 3,
 2,
 2,
 3,
 3,
 3,
 2,
 2,
 5,
 2,
 2,
 6,
 10,
 3,
 11,
 3,
 2,
 2,
 2,
 4,
 3,
 2,
 3,
 3,
 3,
 2,
 3,
 5,
 6,
 4,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 7,
 2,
 2,
 5,
 4,
 2,
 5,
 5,
 2,
 2,
 3,
 2,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 4,
 2,
 2,
 2,
 5,
 8,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 6,
 3,
 2,
 2,
 3,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 4,
 2,
 6,
 2,
 2,
 4,
 2,
 2,
 2,
 3,
 3,
 3,
 2,
 5,
 3,
 4,
 6,
 2,
 3,
 2,
 2,
 2,
 3,
 3,
 1,
 3,
 2,
 2,
 2,
 3,
 3,
 2,
 2,
 3,
 2,
 6,
 3,
 5,
 8,
 18,
 17,
 19,
 6,
 4,
 2,
 4,
 4,
 7,
 8,
 2,
 7,
 15,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 9,
 3,
 5,
 4,
 3,
 2,
 3,
 4,
 2,
 8,
 3,
 19,
 2,
 2,
 5,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 4,
 2,
 4,
 12,
 2,
 6,
 3,
 2,
 5,
 4,
 3,
 3,
 4,
 2,
 7,
 4,
 5,

In [336]:

QAN50 = 0
for size in sorted([len(val) for val in hapBlock.values()], reverse=True):
    QAN50 += size
    
    if QAN50 > phased/2:
        print ('QAN %s' %size)
        break


QAN 3


In [337]:
phased

9469

4734.5